:::{.callout-important}
## 시그모이드 함수 사용 이유

단위 계단 함수(unit step function)은 불연속이므로 미분할 수 없는 지점이 있어 퍼셉트론 학습에 사용하기에 적합하지 않다. 0과 1 사의의 연속적인 값을 가지는 시그모이드(sigmoid) 함수를 사용한다. p214
:::

:::{.callout-important}
다층 퍼셉트론(multi-layer perceptron)에서는 입력 데이터도 하나의 층으로 간주한다. p217
:::

:::{.callout-important}
## 오차 역전파

오차 역전파(back-propagation)는 출력층에서 입력층 방향으로 뒤에서부터 오차를 추적하며 경사 하강법을 이용해 가중치와 편향을 수정하는 수학적 방법이다. 오차 역전파의 수학적 원리는 연쇄 법칙(chain rule)과 편미분(partial derivative)이다. 신경망은 구조가 복잡해서 손실 함수의 기울기를 계산하고 경사 하강법으로 가중치와 편향을 수정하는 과정의 연산량이 많기 때문에 이 연산 과정을 수학적으로 개선한 방법이 오차 역전파다. p218
::: 

:::{.callout-important}
## 기울기 소실(gradient vanishing) 문제

오차 역전파의 계산 과정에서 층마다 활성화 함수의 기울기를 곱하는데, 활성화 함수로 사용하는 시그모이드 함수의 기울기는 0.3 미만이다. 따라서 은닉층을 지날 때마다 0.3 미만의 값을 반복해서 곱하다 보면 입력층에 가까워졌을 때는 기울기가 0에 가까워진다. 가중치를 수정할 때는 기울기의 크기에 학습률을 곱한 만큼 가중치를 수정하게 되는데, 기울기가 0에 가까우면 기울기의 크기에 학습률은 곱한 결과도 0에 가까우므로 가중치가 거의 수정되지 않아 제대로 학습이 되지 않는다. 이를 해결하기 위해 은닉층 활성화 함수로 시그모이드 함수 대신 렐루 함수를 사용한다. 렐루 함수는 값이 0을 넘을 때는 미분 값이 항상 1이기 때문에 많은 은닉층이 있어도 기울기가 소실되지 않고 잘 전달된다.
:::

## 활성화 함수

| 사용층 | 용도 | 활성화 함수 | <span style="display: inline-block; width:400px">설명</span> |
|:-:|:-:|:-:|-|
|은닉층| 기울기 소실 문제를 줄이고 다음 층으로 신호 전달 | 렐루 / 리키 렐루 | 0이하는 모두 0, 0 초과는 항상 1 |
|출력층| 이진 분류 | 시그모이드 | 양성일 확률로 따져 양성, 음성 판정 |
| | 다중 분류 | 소프트맥스 | 각 클래스에 속할 확률을 따져 분류 |
| | 회귀 | - | 노드 값을 그대로 출력 |

## 손실 함수

| 용도 | 손실 함수 |
|:-:|-|
| 이진 분류 | 이진 크로스 엔트로피 |
| 다중 분류 | 크로스 엔트로피 |
| 회귀 | 평균 제곱 오차|

## 옵티마이저(optimizer) - 사용하는 경사 하강법 종류

| 옵티마이저 | 특징 |
|-|-|
| 확률적 경사 하강법(SGD, Stochastic Gradient Descent) | 데이터 샘플을 무작위로 추출하여 일부만 경사 하강법을 사용해 학습 속도 개선|
| RMSProp(Root Mean Square Propagation) | 학습률이 커서 최적 가중치를 지나치는 문제 해결<br>손실 함수의 기울기가 크면 학습률이 크고 작으면 학습률이 작아지도록 조절 |
| 모멘텀(momentum) | 지역 최솟값에 갇히는 문제를 해결하기 위해<br>손실 함수의 기울기가 0인 여러 지점 중 한 지점에서 멈추지 않고<br>계속 최솟값을 찾도록 관성 개념을 추가 |
| 아담(adam) | RMSProp과 모멘텀 기능을 합침 |

:::{.callout-important}
## 텐서(tensor) 자료구조

넘파이 배열과 구조적, 기능적으로 거의 비슷하지만 그래픽 카드를 사용해 병렬 연산을 할 수 있다는 차이가 있다. NVIDIA 그래픽 카드가 제공하는 쿠다(cuda) 환경에서 모델을 학습할 수 있다. 
:::

## 선형 회귀 구현


In [1]:
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim # 옵티마이저

In [3]:
dataset = datasets.load_boston()

X, y = dataset['data'], dataset['target']

### 특성과 타깃을 텐서 자료구조로 변환

- 실수형 텐서: torch.FloatTensor
- 정수형 텐서: torch.LongTensor
- 파이토치는 **타겟을 2차원 배열로 설정**해야 한다.
- 차원 추가 메서드 : unsqueeze() eg. 맨 뒤에 길이가 1인 차원 추가 unsqueeze(-1)
- 차원 제거 메서드 : squeeze

In [4]:
X = torch.FloatTensor(X)
y = torch.FloatTensor(y).unsqueeze(-1)

In [5]:
X.shape, y.shape

(torch.Size([506, 13]), torch.Size([506, 1]))

### 표준화

sklearn의 StandardSacler는 2차원 배열까지만 지원하기 때문에 3차원 이상의 데이터를 많이 다루는 딥러닝에서는 표준화를 직접 하는 것에 익숙해져야 한다.

In [6]:
X = (X - torch.mean(X))/torch.std(X)

### 선형 회귀 모델 객체 생성

(입력값, 출력값) = (특성 13개, 타겟 1개)

- 활성화 함수 => 시그모이드 함수 => 로지스틱 회귀
- 활성화 함수 없는 퍼셉트론 => 선형 회귀

In [7]:
model = nn.Linear(13, 1)

### 손실함수와 옵티마이저 객체 생성

- MSE 손실 함수 -> criterion 변수 이름이 관례
- 확률적 경사 하강법 옵티마이저 
- parameters 메서드로 모델의 가중치를 불러올 수 있다.

In [8]:
criterion = nn.MSELoss()

optimizer = optim.SGD(model.parameters(), lr=0.01)

### 학습 함수 정의(에포크 1회) -> 오차(손실) 반환

1. 손실 함수로 오차 계산(실제 타깃과 모델 예측 타깃의 차이)
1. 미분으로 손실 함수 기울기 계산하여 가중치를 어떻게 수정해야 오차가 줄어드는지 파악
1. 학습률만큼 가중치 수정

한 번의 에포크는 주어진 모든 데이터 샘플을 이용해 한 번 학습하는 것을 의미

In [9]:
def train(model, criterion, optimizer, X, y):
    
    optimizer.zero_grad()           # 기울기 초기화
    hypothesis = model(X)           # 모델 사용해 타깃 예측
    loss = criterion(hypothesis, y) # 오차 계산
    loss.backward()                 # 기울기 계산
    optimizer.step()                # 경사하강법으로 가중치 수정
    return loss.item()              # 현재 에포크 오차 반환 -> 파이썬 숫자형 자료구조로 변환(item)
    

### 학습하며 오차 출력

In [10]:
n_epochs = 100 # 학습 횟수
for epoch in range(1, n_epochs):
    loss = train(model, criterion, optimizer, X, y)
    if epoch % 10 == 0:
        print('epoch: {}, loss: {:.4f}'.format(epoch, loss))

epoch: 10, loss: 115.4186
epoch: 20, loss: 97.9571
epoch: 30, loss: 88.5269
epoch: 40, loss: 82.5146
epoch: 50, loss: 78.6239
epoch: 60, loss: 76.0548
epoch: 70, loss: 74.3105
epoch: 80, loss: 73.0827
epoch: 90, loss: 72.1799


## 로지스틱 회귀

- 이진 크로스 엔트로피 손실 함수 사용
- view 메서드는 넘파이의 reshape와 유사

### 데이터 불러오기, 텐서 변환, 표준화

In [14]:
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim

dataset = datasets.load_breast_cancer()

X, y = dataset['data'], dataset['target']

X = torch.FloatTensor(X)
y = torch.FloatTensor(y).view(-1, 1)

X = (X - torch.mean(X)) / torch.std(X)

X.shape, y.shape

(torch.Size([569, 30]), torch.Size([569, 1]))

### nn.Sequential

Sequential 클래스 생성자에 모델 객체와 활성화 함수 객체를 전달하면 두 객체를 차례대로 사용해 계산한 결과를 반환하는 모델 생성

In [15]:
model = nn.Sequential(
    nn.Linear(30, 1),
    nn.Sigmoid()
)

### 손실함수, 옵티마이저 정의 / 학습 함수 정의 / 학습

이진 크로스 엔트로피(Binary Cross Entropy) 손실함수

In [16]:
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

def train(model, criterion, optimizer, X, y):
    optimizer.zero_grad()
    hypothesis = model(X)
    loss = criterion(hypothesis, y)
    loss.backward()
    optimizer.step()
    return loss.item()

n_epochs = 100
for epoch in range(1, n_epochs + 1):
    loss = train(model, criterion, optimizer, X, y)
    if epoch % 10 == 0:
        print('epoch: {}, loss: {:.4f}'.format(epoch, loss))

epoch: 10, loss: 0.6046
epoch: 20, loss: 0.5323
epoch: 30, loss: 0.4806
epoch: 40, loss: 0.4422
epoch: 50, loss: 0.4126
epoch: 60, loss: 0.3891
epoch: 70, loss: 0.3701
epoch: 80, loss: 0.3543
epoch: 90, loss: 0.3411
epoch: 100, loss: 0.3298


### 타깃 예측 및 평가

In [17]:
y_predicted = (model(X) >= 0.5).float()

score = (y_predicted == y).float().sum()
print('accuracy: {:.2f}'.format(score))

accuracy: 518.00


## 클래스로 모델 정의

- 파이토치에서 모델 클래스를 정의할 때는 nn.Module 클래스를 상속받아야 한다.
- 모델 구조는 `__init__` 생성자 안에 정의한다.

### 선형회귀 모델 클래스 정의

In [18]:
class LinearRegression(nn.Module):
    def __init__(self, num_features): # 생성자에서 모델 구조 정의
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
    
    def forward(self, X): # 순전파 정의
        out = self.linear(X)  # 생성자에서 만든 선형모델로 타깃 예측 및 반환
        return out
    
model = LinearRegression(13)

### 로지스틱 회귀 모델 클래스 정의

In [20]:
class LogisticRegression(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear = nn.Linear(num_features, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, X):
        out = self.linear(X)
        out = self.sigmoid(out)
        return out
    
model = LogisticRegression(30)

## 배치(batch) 학습

전체 데이터에 대한 추론과 오차 역전파를 한 번에 실행하기에는 컴퓨터 메모리 한계가 존재한다. 이 문제를 해결하기 위해 데이터를 나눠서 처리하는 배치라는 개념이 등장한다. 배치란 한 번의 에포크를 수행할 때 처리하는 샘플의 개수를 의미한다. 전체 데이터 샘플을 지정된 배치 크기로 나눠서 학습하고 마지막 배치 학습이 끝나면 한 번의 에포크가 수행된 것이다.

배치 학습을 위해 DataLoader 클래스 제공

### 데이터 로드, 표준화

In [24]:
from sklearn import datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

dataset = datasets.load_breast_cancer()
X, y = dataset['data'], dataset['target']
X = torch.FloatTensor(X)
y = torch.FloatTensor(y).view(-1,1)

X = (X - torch.mean(X)) / torch.std(X)

### 배치 학습 구현 위해 데이터와 타깃 합치기



In [25]:
# 입력 데이터와 타깃을 묶어 텐서 데이터셋 생성
dset = TensorDataset(X, y)

# 한 번에 256개의 데이터 샘플을 배치로 사용하는 데이터로더 생성
loader = DataLoader(dset, batch_size=256, shuffle=True)

### 신경망 모델 클래스 정의 / 학습 함수 정의 / 학습 / 평가

In [29]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.linear1 = nn.Linear(num_features, 4) # 은닉층 노드 4개 생성
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(4, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, X):
        out = self.linear1(X)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

model = NeuralNetwork(30) 
criterion = nn.BCELoss()  # 이진 크로스 엔트로피
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 학습 함수 정의
def train(model, criterion, optimizer, loader):
    epoch_loss = 0    # 현재 에포크의 오차 저장 변수
    
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item() # 현재 배치의 오차 누적
        
    return epoch_loss / len(loader) # 현대 에포크의 오차 평균 반환

# 모델 학습
n_epochs = 100
for epoch in range(1, n_epochs + 1):
    loss = train(model, criterion, optimizer, loader)
    if epoch % 10 == 0:
        print('epoch: {}, loss: {:.4f}'.format(epoch, loss))
        
# 학습된 모델로 타깃 예측
y_predicted = (model(X) >= 0.5).float()

# 정확도 계산
score_trained_model = (y_predicted == y).float().mean()
print('accuracy: {:.2f}'.format(score_trained_model))

epoch: 10, loss: 0.5338
epoch: 20, loss: 0.3442
epoch: 30, loss: 0.2628
epoch: 40, loss: 0.2579
epoch: 50, loss: 0.2114
epoch: 60, loss: 0.1996
epoch: 70, loss: 0.2114
epoch: 80, loss: 0.2221
epoch: 90, loss: 0.2065
epoch: 100, loss: 0.2161
accuracy: 0.92


### 모델 저장

- model.state_dict() 모델 정보를 가진 딕셔너리에 접근
- torch.save() 모델 저장 .pt 또는 .pth

In [30]:
torch.save(model.state_dict(), 'breast_cancer.pt')

### 모델 로드, 예측

- torch.load() 모델 불러오기

In [31]:
# 신경망 모델의 객체 생성
loaded_model = NeuralNetwork(30)

# 저장한 파일에서 가중치 불러와 복원
loaded_model.load_state_dict(torch.load('breast_cancer.pt'))

# 타깃 예측
y_pred2 = (loaded_model(X) >= 0.5).float()

score_loaded_model = (y_pred2 == y).float().mean()
print('accuracy of loaded model: {:.2f}'.format(score_loaded_model))

accuracy of loaded model: 0.92


## 심층 신경망(DNN) = 전결합(fully connected) 신경망

전결합층(fully connected layer) : 이전 층의 모든 노드와 연결된 노드로 구성된 은닉층 또는 츨력층

:::{.callout-important}
DNN은 1차원 배열을 입력으로 받는다.
:::

### 손글씨 이미지 분류

MNIST datasets는 torchvision 라이브러리에 내장

#### 데이터 불러오기

In [32]:
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

# 현재 경로에 MNIST 학습 세트와 테스트 세트 불러오기
path = './'
train_dataset = datasets.MNIST(path, train=True, download=True)
test_dataset = datasets.MNIST(path, train=False, download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



#### 입력 데이터를 1차원으로 변환

In [49]:
# X, y로 나누고 정규화. 0~255 사이의 흑백이미지 -> 0~1 사이의 실수값
# 정규화를 하면 오파를 줄이기 위한 에포크 반복 횟수를 줄일 수 있다.
X_train, y_train = train_dataset.data / 255, train_dataset.targets
X_test, y_test = test_dataset.data / 255, test_dataset.targets

# train, test 데이터 형태 확인
# 학습 6만 개, 테스트 만 개
print('학습 세트 입력 데이터:', X_train.shape)
print('학습 세트 타겟:', y_train.shape)
print('테스트 세트 입력 데이터:', X_test.shape)
print('테스트 세트 타겟:', y_test.shape)
print()

#########################################################################
# 전결합층은 1차원 벡터를 입력값으로 받기 때문에 1차원 배열로 변환. 이미지 크기 28*28 = 784
#########################################################################
X_train, X_test = X_train.view(-1, 784), X_test.view(-1, 784)
print('학습 세트 입력 데이터:', X_train.shape)
print('테스트 세트 입력 데이터:', X_test.shape)

################################################
# 배치 학습 위해 입력 데이터와 타깃을 묶어 텐서데이터세트 생성
################################################
train_dset = TensorDataset(X_train, y_train)
test_dset = TensorDataset(X_test, y_test)

# 한 번에 32개의 데이터 샘플을 배치로 사용하는 데이터로더 생성
train_loader = DataLoader(train_dset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=True)

######################
# DNN 모델 클래스 정의
######################
class DNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()

        # 첫 번째 은닉층
        self.hidden_layer1 = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU()
        )
        
        # 두 번째 은닉층
        self.hidden_layer2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU()
        )
        
        # 출력층 정의 : 0부터 9까지 숫자를 구분 -> 노드 개수 10
        # 다중 분류는 출력층에 softmax 함수를 사용하지만 
        # 사용할 크로스 엔트로피 손살 함수에 소프트맥스 연산이 내장되어 있어 생략
        self.output_layer = nn.Linear(128, 10)
        
    def forward(self, X):
        out = self.hidden_layer1(X)
        out = self.hidden_layer2(out)
        out = self.output_layer(out)
        return out
    
##############################
# 그래픽카드 사용 가능할 경우 이용한다.
##############################
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 784개 값을 입력받는 로지스틱 회귀 모델 객체 생성
# 모델 객체와 데이터세트틑 to 메서드를 이용해 어떤 장치에서 연산할지 선택
model = DNN(784).to(device)

# 손실함수와 옵티마이저 객체 생성
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

##############
# 학습 함수 정의
##############
def train(model, criterion, optimizer, loader):
    '''현대 에포크의 손실(오차)과 정확도 반환'''
    
    epoch_loss = 0 # 현재 에포크 오차
    epoch_acc = 0  # 현재 에포크 정확도
    
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad() # 기울기 초기화
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward() # 기울기 계산
        optimizer.step() # 가중치 수정
        y_predicted = torch.argmax(hypothesis, 1) # torch.argmax(input, dim)
        acc = (y_predicted == y_batch).float().mean() # 정확도
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(loader), epoch_acc / len(loader)

##############################
# 평가 함수 정의 - test data 사용
##############################
def evaluate(model, criterion, loader):
    
    epoch_loss = 0
    epoch_acc = 0
    
    with torch.no_grad(): # 오차 역전파가 사용되지 않도록 파이토치가 연산 내역을 추적하지 않게 한다
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            hypothesis = model(X_batch)
            loss = criterion(hypothesis, y_batch)
            y_predicted = torch.argmax(hypothesis, 1)
            acc = (y_predicted == y_batch).float().mean()
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
    return epoch_loss / len(loader), epoch_acc / len(loader)

###############
# 20회 학습 실행
###############
n_epochs = 20
for eppoch in range(1, n_epochs+1):
    
    # 모델 학습
    loss, acc = train(model, criterion, optimizer, train_loader)
    
    # 모델 평가
    test_loss, test_acc = evaluate(model, criterion, test_loader)
    
    print('epoch: {}, loss: {:.3f}, acc: {:.2f}, test_loss: {:.3f}, test_acc: {:.3f}'.format(epoch, loss, acc, test_loss, test_acc))

학습 세트 입력 데이터: torch.Size([60000, 28, 28])
학습 세트 타겟: torch.Size([60000])
테스트 세트 입력 데이터: torch.Size([10000, 28, 28])
테스트 세트 타겟: torch.Size([10000])

학습 세트 입력 데이터: torch.Size([60000, 784])
테스트 세트 입력 데이터: torch.Size([10000, 784])
epoch: 100, loss: 0.548, acc: 0.86, test_loss: 0.276, test_acc: 0.922
epoch: 100, loss: 0.254, acc: 0.93, test_loss: 0.215, test_acc: 0.938
epoch: 100, loss: 0.203, acc: 0.94, test_loss: 0.182, test_acc: 0.947
epoch: 100, loss: 0.169, acc: 0.95, test_loss: 0.159, test_acc: 0.953
epoch: 100, loss: 0.143, acc: 0.96, test_loss: 0.138, test_acc: 0.959
epoch: 100, loss: 0.123, acc: 0.96, test_loss: 0.120, test_acc: 0.964
epoch: 100, loss: 0.106, acc: 0.97, test_loss: 0.108, test_acc: 0.967
epoch: 100, loss: 0.093, acc: 0.97, test_loss: 0.103, test_acc: 0.969
epoch: 100, loss: 0.081, acc: 0.98, test_loss: 0.095, test_acc: 0.971
epoch: 100, loss: 0.071, acc: 0.98, test_loss: 0.086, test_acc: 0.975
epoch: 100, loss: 0.063, acc: 0.98, test_loss: 0.082, test_acc: 0.975
epoc

### 과적합 줄이기 - dropout

- 가중치 수가 아주 많으면 학습 세트의 입력 데이터와 타겟을 완벽하게 매핑하도록 가중치를 학습 -> 일반화 오류 가능성 커짐
- 가중치 수가 적으면 많은 양의 정보를 저장할 수 없으므로 전체 데이터 샘플의 일반적 특징을 학습
- 따라서, 과적합을 줄이고 일반화를 높이려면 가중치 수를 줄여 모델을 단순하게 만들면 된다.
- 은닉층 또는 은닉층의 노드 수를 수정하거나 조기 종료
- 드롭아웃(dropout) : 무작위로 일부 노드를 누락시켜 특정 정보만으로 결론을 도출하지 못하도록 규제하는 학습 방법이다. eg. 머리 짧은 남자와 머리 긴 여자 사진만 학습하면 머리가 길면 무조건 여자로 판단할 수 있어, 드롭아웃을 적용하면 사진의 일부로 조금씩 가려 학습시킨다. 
- 드롭아웃은 매번 지정된 비율만큼의 노드를 무작위로 누락시켜 값을 0으로 만든다. 


## CNN(Convolutional Neural Network, 합성곱 신경망)

- 컴퓨터 비전 문제에서 머신러닝에 비해 좋은 성능을 보여줌
- 이미지를 여러 조각으로 쪼개 지역적인 특징 추출
- 출력층은 전결합층을 사용하는 경우 많음
- 보통 입력 데이터보다 작은 크기의 필터를 입력 데이터 영역에서 이동시키며 특성 맵(feature map)이라고 하는 값은 계산함. 필터의 크기가 클수록 데이터 영역에서 움직이는 횟수가 적어 특성 맵 크기는 작아진다.
- 하나의 합성곱층에서 여러 개의 필터를 사용해 여러 개의 특성맵을 생성하는데, 각각의 특성맵은 이미지의 모서리나 질감 등 다양한 특징을 발견한다.
- 합성곱층을 여러 겹 쌓을수록 고수준의 특징을 감지하는 특성 맵을 만들 수 있다. 초반에는 모서리나 질감 같은 저수준의 특징을 잡아내고 이후 층으로 갈수록 저수준의 특징을 이용해 눈, 코, 입 같은 고수준의 특징을 감지한다.

### CNN 활용한 손글씨 이미지 분류

:::{.callout-important}
합성곱 신경망은 **채널, 높이, 너비 순서로 3차원 배열을 입력**으로 받는다.
:::

MNIST 데이터세트는 흑백 이미지이므로 채널이 한 개이지만, 실제로는 채널을 생략하고 높이와 너비만으로 구성된 2차원 데이터이다. 따라서 unsqueeze 메서드로 높이 차원 앞에 채널 차원을 추가하여 3차원으로 변환한다.

In [67]:
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

# 현재 경로에 MNIST 학습 세트와 테스트 세트 불러오기
path = './'
train_dataset = datasets.MNIST(path, train=True, download=True)
test_dataset = datasets.MNIST(path, train=False, download=True)

X_train, y_train = train_dataset.data / 255, train_dataset.targets
X_test, y_test = test_dataset.data / 255, test_dataset.targets

print('train features:', X_train.shape)
print('train target:', y_train.shape)
print('test fearues:', X_test.shape)
print('test target:', y_test.shape)

############################
# 채널을 추가하여 3차원 배열로 변환
############################
X_train, X_test = X_train.unsqueeze(1), X_test.unsqueeze(1)
print('\ntrain features:', X_train.shape)
print('test features:', X_test.shape)

# 배치 
train_dset = TensorDataset(X_train, y_train)
test_dset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=True)

####################
# CNN 모델 클래스 정의
####################
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 합성곱 은닉층
        self.hidden_layer1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=(3,3)), # Conv2d(채널수, 필터수) -> 64개 특성맵 생성
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)), # 최대 풀링 -> 학습되는 가중치 없음
            nn.Dropout(0.5)
        )
        
        # 합성곱 은닉층
        self.hidden_layer2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3,3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2,2)),
            nn.Dropout(0.5)
        )
        
        # 전결층 -> 1차원으로 변환
        self.hidden_layer3 = nn.Linear(128*5*5, 128) # 이전 은닉층에서 높이 5, 너비 5인 특성 맵 128개 생성
        nn.ReLU()
        
        # 출력층 -> 최종 분류는 10개
        self.output_layer = nn.Linear(128, 10)
        
    def forward(self, X):
        out = self.hidden_layer1(X)
        out = self.hidden_layer2(out)
        out = out.view(out.shape[0], -1) # 전결층 -> 1차원으로 변환
        out = self.hidden_layer3(out)
        out = self.output_layer(out)
        
        return out
        
# 그래픽 카드 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 합성곱 신경망 모델 객체 생성
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# 학습 함수 정의
def train(model, criterion, optimizer, loader):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() # 모델을 학습 모드로 설정
    
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        y_predicted = torch.argmax(hypothesis, 1)
        acc = (y_predicted == y_batch).float().mean()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(loader), epoch_acc / len(loader)
    
# 평가 함수 정의
def evaluate(model, criterion, loader):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval() # 모델을 평가 모드로 설정
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            hypothesis = model(X_batch)
            loss = criterion(hypothesis, y_batch)
            y_predicted = torch.argmax(hypothesis, 1)
            acc = (y_predicted == y_batch).float().mean()
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
        return epoch_loss / len(loader), epoch_acc / len(loader)
    
# 모델 학습
n_epochs = 20
for epoch in range(1, n_epochs+1):
    loss, acc = train(model, criterion, optimizer, train_loader)
    test_loss, test_acc = evaluate(model, criterion, test_loader)
    
    print('epoch: {}, loss: {:.3f}, acc: {:.3f}, test_loss: {:.3f}, test_acc: {:.3f}'.format(epoch, loss, acc, test_loss, test_acc))


train features: torch.Size([60000, 28, 28])
train target: torch.Size([60000])
test fearues: torch.Size([10000, 28, 28])
test target: torch.Size([10000])

train features: torch.Size([60000, 1, 28, 28])
test features: torch.Size([10000, 1, 28, 28])
epoch: 1, loss: 0.427, acc: 0.87, test_loss: 0.127, test_acc: 0.963
epoch: 2, loss: 0.138, acc: 0.96, test_loss: 0.072, test_acc: 0.978
epoch: 3, loss: 0.101, acc: 0.97, test_loss: 0.057, test_acc: 0.982
epoch: 4, loss: 0.086, acc: 0.97, test_loss: 0.046, test_acc: 0.985
epoch: 5, loss: 0.076, acc: 0.98, test_loss: 0.042, test_acc: 0.987
epoch: 6, loss: 0.069, acc: 0.98, test_loss: 0.040, test_acc: 0.987
epoch: 7, loss: 0.064, acc: 0.98, test_loss: 0.036, test_acc: 0.989
epoch: 8, loss: 0.060, acc: 0.98, test_loss: 0.035, test_acc: 0.989
epoch: 9, loss: 0.058, acc: 0.98, test_loss: 0.033, test_acc: 0.989
epoch: 10, loss: 0.053, acc: 0.98, test_loss: 0.034, test_acc: 0.989
epoch: 11, loss: 0.052, acc: 0.98, test_loss: 0.030, test_acc: 0.990
epo

### 가중치 수 계산

torchsummary.summary

In [69]:
# !pip install torchsummary
from torchsummary import summary

summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 26, 26]             640
              ReLU-2           [-1, 64, 26, 26]               0
         MaxPool2d-3           [-1, 64, 13, 13]               0
           Dropout-4           [-1, 64, 13, 13]               0
            Conv2d-5          [-1, 128, 11, 11]          73,856
              ReLU-6          [-1, 128, 11, 11]               0
         MaxPool2d-7            [-1, 128, 5, 5]               0
           Dropout-8            [-1, 128, 5, 5]               0
            Linear-9                  [-1, 128]         409,728
           Linear-10                   [-1, 10]           1,290
Total params: 485,514
Trainable params: 485,514
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.11
Params size (MB): 1.85
Estimated T

### shape 계산

1. Conv2d 후 특성 맵 shape 계산

$\text{특성맵의 행 크기} = \frac{\text{입력 데이터의 행 크기} - \text{필터의 행 크기}}{stride} + 1$

$\text{특성맵의 열 크기} = \frac{\text{입력 데이터의 열 크기} - \text{필터의 열 크기}}{stride} + 1$

- 입력(1, 28, 28) 채널 1개, 높이 28, 넓이 28, 필터(64개, 3, 3)
- stride는 조건이 없으니 1로 가정
- (28 - 3) + 1 => 26 
- 따라서 (64, 26, 26) 26x26 크기의 특성 맵이 64개

2. ReLU 활성화 함수에서 학습되는 가중치는 없고 데이터 형태에도 영향을 주지 않는다.

3. Pooling 2x2 -> 맵의 크기가 절반으로 줄어듬. 채널 영향은 없음

(64, 26, 26) -> (64, 13, 13)

4. 두 번째 은닉층

- 입력되는 채널은 64개 -> 테이터의 채널 수 만큼 필터의 채널 수를 늘림
- 합성곱층에 입력되는 데이터 채널 수 에 관계없이 필터 1개당 1개의 채널을 가진 특성 맵을 출력한다. 
- 두 번째 합성곱층의 필터 크기는 64x3x3 = 576 + 1개 편향 => 필터 1개당 가중치 577개
- 필터 수 128개로 지정 => 총 파라미터 개수는 577 * 128 = 73,856
- 생성한 맵은 (128, 11, 11)

5. 풀링 (11, 11) -> (5.5, 5.5) -> 소수점 버림 -> (128, 5, 5)

6. 세 번째 은닉층은 128개 노드를 가진 전결합층 -> 1차원 배열 입력 : 128 * 5 * 5 = 3,200

- 입력 데이터는 모든 노드와 연결되므로 입력 3200 * 노드 128 + 편향 128 = 409,728

7. 출력층 

- 전결합층은 클래스 개수와 같은 10개 노드
- 입력 128 * 노드 10 + 편향 10 = 1290

### stride

유의할 점은 합성곱층에 비해 전결합층에서 가중치가 많아지기 쉽다는 것이다. 전결합층은 입력 데이터가 모든 노드와 연결되기 때문에 입력 데이터의 크기가 조금만 커져도 가중치 수가 급격히 늘어난다. 따라서 전결합층을 사용하기 전에 특성 맵의 크기를 줄여야 하는데, 가중치를 줄이는 방법으로 풀링 외에 스트라이드(stride)가 있다.

### 제로 패딩(zero padding)

풀링층을 사용하지 않고 스트라이드도 기본값을 두더라도 합성곱층을 거치면서 특성 맵의 크기는 조금씩 줄어든다. 특성 맵의 크기가 계속 작아지면 합성곱층을 많이 배치하지 못하게 된다. 이럴 때는 테두리에 값이 0인 픽셀을 추가해서 원본 이미지의 크기를 키운 후 특성 맵을 도출한다. 

## 전이 학습(transfer learning)

- VGG16 모델이 가진 세 개의 전결합층 중에서 출력층만 새로운 전결합층으로 교체
- 합성곱층의 가중치는 수정되지 않도록 고정하고 전결합층의 가중치만 수정해서 모델 학습
- datasets 모듈의 ImageFolder 클래스: 폴더 안이 이미지 불러오기
- torchvision의 transforms 모듈 : 이미지의 크기나 밝기 등의 데이터 변환 -> 데이터 증강(data augmentation) -> 일반화 성능 높임

AdaptiveAvgPool2d(적응형 평균 풀링)

- 필터의 크기를 직접 지정하는 것이 평균 풀링이라면
- 특성 맵의 크기를 지정해서 지정한 크기의 특성 맵이 생성되도록 피터의 크기를 자동으로 조절하는 것을 적응형 풀링이라고 한다.

In [ ]:
# !wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
# !unzip cats_and_dogs_filtered.zip data/

In [91]:
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

#############################
# 데이터 변환
#############################
train_config = transforms.Compose([transforms.Resize((224, 224)),
                                  transforms.RandomHorizontalFlip(),  # 이미지를 무작위로 좌우반전
                                  transforms.ToTensor()])

test_config = transforms.Compose([transforms.Resize((224,224)),
                                  transforms.ToTensor()])

# 이미지를 불러와 위 설정을 반영한 데이터세트 자료구조 만들기
train_dset = datasets.ImageFolder('data/cats_and_dogs_filtered/train/', train_config)
test_dset = datasets.ImageFolder('data/cats_and_dogs_filtered/validation/', test_config)

# 한 번에 32개의 데이터 샘플을 배치로 사용하는 데이터로더 생성
train_loader = DataLoader(train_dset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=True)

# 사전 학습 모델인 VGG16 모델 객체 생성 후 가중치 불러옴
model = models.vgg16(pretrained=True)

# 모델 구조 확인
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [92]:
####################################
# 모델의 가중치를 더이상 학습하지 않도록 설정
####################################
for param in model.features.parameters():
    param.require_grad = False
    
# 출력층을 한 개의 노드를 가진 전결합층으로 교체 -> 마지막 층의 out_features = 1로 변경
model.classifier[-1] = nn.Sequential(
    nn.Linear(model.classifier[-1].in_features, 1),
    nn.Sigmoid()
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(model.to(device))

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [93]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-6)

def train(model, criterion, optimizer, loader):
    epoch_loss = 0
    epoch_acc = 0

    model. train()

    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1,1)
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        y_predicted = hypothesis >= 0.5
        acc = (y_predicted == y_batch).float().mean()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    return epoch_loss / len(loader), epoch_acc / len(loader)

def evaluate(model, criterion, loader):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
            hypothesis = model(X_batch)
            loss = criterion(hypothesis, y_batch)
            y_predicted = hypothesis >= 0.5
            acc = (y_predicted == y_batch).float().mean()
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(loader), epoch_acc / len(loader)

n_epoch = 10
for epoch in range(1, n_epoch+1):
    loss, acc = train(model, criterion, optimizer, train_loader)
    test_loss, test_acc = evaluate(model, criterion, test_loader)

    print('epoch: {}, loss: {:.3f}, acc: {:.3f}, test_loss: {:.3f}, test_acc: {:.3f}'.format(epoch, loss, acc, test_loss, test_acc))

epoch: 1, loss: 0.712, acc: 0.536, test_loss: 0.622, test_acc: 0.698
epoch: 2, loss: 0.572, acc: 0.758, test_loss: 0.492, test_acc: 0.881
epoch: 3, loss: 0.428, acc: 0.878, test_loss: 0.334, test_acc: 0.938
epoch: 4, loss: 0.274, acc: 0.933, test_loss: 0.196, test_acc: 0.958
epoch: 5, loss: 0.175, acc: 0.950, test_loss: 0.126, test_acc: 0.965
epoch: 6, loss: 0.118, acc: 0.965, test_loss: 0.093, test_acc: 0.972
epoch: 7, loss: 0.092, acc: 0.971, test_loss: 0.075, test_acc: 0.974
epoch: 8, loss: 0.080, acc: 0.971, test_loss: 0.066, test_acc: 0.976
epoch: 9, loss: 0.065, acc: 0.974, test_loss: 0.065, test_acc: 0.974
epoch: 10, loss: 0.061, acc: 0.980, test_loss: 0.055, test_acc: 0.977


## RNN(Recurrent Neural Network, 순환 신경망)

- 순서가 중요한 연속적인 데이터 처리에 많이 사용. 언어 번역, 자동 완성, 날씨 예보

### 기온, 풍속, 습도 이용해 24시간 뒤의 기온 추론

- 입력 데이터는 24시간 동안 3가지 기상 정보가 한 시간 단위로 저장 -> (24, 3)
- 타겟은 24시간 뒤의 기온 정보 -> (1,)

nn.RNN(특성 개수, 출력할 데이터 특성 수, 첫번째 차원 배치 처리 여부)

- 순환 신경망은 첫 번째 차원이 타입 스텝 정보를 나타내는 시퀀스이기 때문에 batch_first=True 하여 배치로 변경

cell(X) -> 2개 반환 <br>
① 24개의 타입 스텝에 대한 셀의 출력값(24, 3) -> 전결합층에 전달<br>
② 마지막 타임 스텝에 대한 셀의 출력값(1, 3) 

contiguous

- batch_first=True로 배치를 첫 번째 차원으로 처리하도록 설정함으로써 셀에 출력된 결과물의 첫 번째 차원이 배치인 것처럼 보이지만 메모리 상에서는 여전히 배치가 두 번째 차원에 존재한다. 이렇게 데이터가 실제로 메모리에 저장된 구조와 차이가 있으면 view 메서드로 데이터의 형태를 변경할 수 없다. 따라서 contiguous 메서드로 배치가 첫 번째 차원인 데이터를 메모리에 새로 만들고, 그 후에 view 메서드로 데이터를 1차원 배열 형태로 변경한다.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import joblib

datasets = joblib.load('weather.pickle')

data, target = datasets['data'], datasets['target']

print('특성과 타겟 데이터 형태:', data.shape, target.shape)
print('특성 일부 데이터 보기', data[0])

# 학습, 테스트 나누기
train_length = 20000
X_train, X_test = data[:train_length], data[train_length:]
y_train, y_test = target[:train_length], target[train_length:]

# 텐서 구조로 변환
X_train, X_test = torch.from_numpy(X_train), torch.from_numpy(X_test)
y_train, y_test = torch.from_numpy(y_train), torch.from_numpy(y_test)

dset_train, dset_test = TensorDataset(X_train, y_train), TensorDataset(X_test, y_test)

train_loader = DataLoader(dset_train, batch_size=256, shuffle=True)
test_loader = DataLoader(dset_test, batch_size=256, shuffle=True)

#####################
# RNN 모델 클래스 정의
#####################
class RNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cell = nn.RNN(3, 3, batch_first=True)
        self.fc = nn.Linear(24*3, 1) # 최종 기온 1개 노드

    def forward(self, X):
        out, hidden_state = self.cell(X)
        out = out.contiguous()
        out = self.fc(out.view(-1, 24*3))
        return out

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = RNN().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=2e-4)

def train(model, criterion, optimizer, loader):
    epoch_loss = 0

    model.train()

    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(loader)

def evaluate(model, criterion, loader):
    epoch_loss = 0

    model.eval()

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
            hypothesis = model(X_batch)
            loss = criterion(hypothesis, y_batch)
            epoch_loss += loss.item()

    return epoch_loss / len(loader)

for epoch in range(1, 201):
    loss = train(model, criterion, optimizer, train_loader)
    test_loss = evaluate(model, criterion, test_loader)

    if epoch % 20 == 0:
        print('epoch: {}, loss: {:.3f}, test_loss: {:.3f}'.format(epoch, loss, test_loss))


In [ ]:
model.eval()

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
        hypothesis = model(X_batch)
        print('predicted target: {:.2f}, real target: {:.2f}'.format(hypothesis[0].item(), y_batch[0].item()))

## LSTM(Long Short-Term Memory)

RNN도 타임 스텝을 많이 반복하면 기울기 소실 문제 발생. 이를 개선한 모델이 LSTM이다.

워드 임베딩(word embedding) : 학습을 통해 밀집 벡터(dense vector)를 얻는 과정. 

파이토치에서는 단어를 밀집 벡터로 표현하기 위해 임베딩층을 사용한다. 처음에는 모든 단어가 무작위 값을 가지는 밀집 벡터로 표현되고, 학습이 진행되면서 역전파를 통해 점차 밀집 벡터가 모델의 추론 성능에 도움이 되는 값, 즉 단어의 이미를 더 잘 표현하는 값으로 조정된다.



In [1]:
from torch.hub import load
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab
from torchtext.datasets import IMDB
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from collections import Counter # 데이터세트에 등장하는 단어들의 출현 빈도
import torch
import torch.nn as nn
import torch.optim as optim

# 학습 세트만 먼저 불러온 이유는 단어장을 만들기 위해
train_dataset = IMDB(split='train')
tokenizer = get_tokenizer('basic_english')
counter = Counter()

# 학습 세트의 문장을 단어 단위로 토큰화하고 단어별 누적 사용 빈도 계산
for label, text in train_dataset:
    # 문장을 단어 단위로 토큰화하고 단어별로 사용 빈도 기록
    counter.update(tokenizer(text))

# 10번 이상 사용된 단어로 단어장을 만든다
vocabulary = vocab(counter, min_freq=10)
vocabulary.set_default_index(0)  # 단어장에 없는 단어는 0으로 설정

# 텍스트를 정수 인코딩하는 람다 함수 정의
text_transform = lambda x: [vocabulary[token] for token in tokenizer(x)]

# 레이블을 정수값으로 치환하는 람다 함수 정의
# pos = 1, neg = 0 으로 치환
label_transform = lambda x: 1 if x == 'pos' else 0 

# 두 개의 람다 함수를 이용해 텍스트와 레이블을 전처리하는 함수 정의
def preprocessing(batch):
    label_list, text_list = [], []
    # 람다 함수로 배치값을 차례대로 변환
    for (_label, _text) in batch:
        # 레이블에 람다 함수 적용
        label_list.append(label_transform(_label))
        # 텍스트에 람다 함수 적용
        text_list.append(torch.tensor(text_transform(_text)))

    # 정수 인코딩된 데이터 길이는 문장에서 몇 개의 단어가 사용되었는지에 따라 달라지기 때문에
    # pad_sequence 함수를 이용해 가장 긴 문장을 기준으로 정수 인코딩된 문장의 길이 통일
    # 길이가 가장 긴 데이터를 기준으로 나머지 문장들은 패딩값을 0으로 채워 데이터 길이 통일
    data = pad_sequence(text_list)
    target = torch.tensor(label_list)
    return data, target

train_dataset, test_dataset = IMDB(split=('train', 'test'))

# preprocessing 함수를 적용하여 학습 세트 데이터로더와 데이터 세트 데이터로더 만듦
train_loader = DataLoader(list(train_dataset), batch_size=8, 
                          shuffle=True, collate_fn=preprocessing)
test_loader = DataLoader(list(test_dataset), batch_size=8,
                         shuffle=False, collate_fn=preprocessing)

# LSTM 모델 클래스 정의
class LSTM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # 모델 구조 정의
        self.embed = nn.Embedding(vocab_size, 16)
        self.cell = nn.LSTM(16, 16)
        self.fc = nn.Linear(16, 1) # 긍정, 부정 이진분류
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        out = self.embed(X)
        out, (hidden_state, cell_state) = self.cell(out)
        out = self.fc(hidden_state.view(-1, 16))
        out = self.sigmoid(out)
        return out

device = 'cuda' if torch.cuda.is_available() else 'cpu'

vocab_size = len(vocabulary)
model = LSTM(vocab_size).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

def train(model, criterion, optimizer, loader):
    epoch_loss = 0
    epoch_loss = 0

    model.train()

    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        acc = ((hypothesis >= 0.5) == y_batch).float().mean()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    
    return epoch_loss / len(loader), epoch_acc / len(loader)

def evaluate(model, criterion, loader):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
            hypothesis = model(X_batch)
            loss = criterion(hypothesis, y_batch)
            acc = ((hypothesis >= 0.5) == y_batch).float().mean()
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    
    return epoch_loss / len(loader), epoch_acc / len(loader)
                                                     
n_epochs = 25
for epoch in range(1, n_epochs+1):
    loss, acc = train(model, criterion, optimizer, train_loader)
    test_loss, test_acc = evaluate(model, criterion, test_loader)

    print('epoch: {}, loss: {:.3f}, acc: {:.3f}, test_loss: {:.3f}, test_acc: {:.3f}'.format(epoch, loss, acc, test_loss, test_acc))

ModuleNotFoundError: No module named 'torchtext'